In [82]:
import pandas as pd
import yfinance as yf
import numpy as np
import pandas_ta as ta
import matplotlib.pyplot as plt
from matplotlib import rcParams
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import minimize
# import ace_tools as tools; 

# 配置中文字体
plt.rcParams['font.family'] = 'Heiti TC'

# plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['figure.dpi'] = 150
plt.rcParams.update({'font.size': 12})

In [83]:
def get_stock_data(stock_symbols, start_date, end_date):
    stock_data = {}
    for symbol in stock_symbols:
        stock_data[symbol] = yf.download(symbol, start=start_date, end=end_date)
    return stock_data

def get_stock_price(stock_symbols, start_date, end_date):
    stock_data = {}
    for symbol in stock_symbols:
        stock_data[symbol] = yf.download(symbol, start=start_date, end=end_date)['Adj Close']
    return pd.DataFrame(stock_data)

def calculate_moving_average(stock_prices, short_window, long_window):
    short_ma = stock_prices.rolling(window=short_window).mean()
    long_ma = stock_prices.rolling(window=long_window).mean()
    return short_ma, long_ma

# Calculate Z-score for mean reversion
def calculate_z_score(stock_prices, window):
    rolling_mean = stock_prices.rolling(window=window).mean()
    rolling_std = stock_prices.rolling(window=window).std()
    z_score = (stock_prices - rolling_mean) / rolling_std
    return z_score

# Adjust Threshold Dynamically
def dynamic_threshold(SSINDX, window):
    std_SSINDX = (SSINDX - SSINDX.mean())/SSINDX.std()
    rolling_std = std_SSINDX.rolling(window).std()
    return 2.5 + 2 * rolling_std

# Portfolio optimization
def portfolio_optimization(returns, cov_matrix):
    num_assets = len(returns)

    def objective(weights):
        return np.dot(weights.T, np.dot(cov_matrix, weights))
    
    def constraint(weights):
        return np.sum(weights) - 1

    cons = ({'type': 'eq', 'fun': constraint})
    bounds = tuple((0, 1) for _ in range(num_assets))
    result = minimize(objective, num_assets * [1. / num_assets,], bounds=bounds, constraints=cons)
    
    return result.x

# Calculate max drawdown
def calculate_max_drawdown(portfolio_values):
    cumulative_max = np.maximum.accumulate(portfolio_values)
    drawdown = (cumulative_max - portfolio_values) / cumulative_max
    max_drawdown = np.max(drawdown) * 100
    return max_drawdown

# Summarize PnL analysis
def pnl_summary(portfolio_values, dates, buy_count, sell_count):
    df = pd.DataFrame({'Date': dates, 'Portfolio Value': portfolio_values})
    df.set_index('Date', inplace=True)
    df['Return'] = df['Portfolio Value'].pct_change()
    df['Year'] = df.index.year

    summary = df.groupby('Year').agg({
        'Portfolio Value': ['first', 'last'],
        'Return': ['std']
    })
    summary.columns = ['Start Value', 'End Value', 'Return Std']
    summary['Annual Return %'] = ((summary['End Value'] / summary['Start Value']) - 1) * 100
    summary['Sharpe Ratio'] = summary['Annual Return %']  / (summary['Return Std'] * np.sqrt(252) * 100)
    
    # Calculate Max Drawdown per year
    max_drawdowns = []
    for year in summary.index:
        year_values = df[df['Year'] == year]['Portfolio Value']
        max_drawdowns.append(calculate_max_drawdown(year_values.values))

    summary['Max Drawdown %'] = max_drawdowns
    summary['Buy Count'] = buy_count
    summary['Sell Count'] = sell_count
    
    # Round to 2 decimal places
    summary = summary.round(2)

    return summary

# Summarize total PnL analysis
def pnl_summary_total(portfolio_values, dates):
    df = pd.DataFrame({'Date': dates, 'Portfolio Value': portfolio_values})
    df.set_index('Date', inplace=True)
    df['Return'] = df['Portfolio Value'].pct_change()

    total_return = ((df['Portfolio Value'][-1] / df['Portfolio Value'][0]) - 1) * 100
    sharpe_ratio = df['Return'].mean() / df['Return'].std() * np.sqrt(252)
    max_drawdown = calculate_max_drawdown(df['Portfolio Value'].values)

    summary_total = pd.DataFrame({
        'Total Return %': [total_return],
        'Sharpe Ratio': [sharpe_ratio],
        'Max Drawdown %': [max_drawdown]
    })

    # Round to 2 decimal places
    summary_total = summary_total.round(2)

    return summary_total

In [84]:
#### 按照排序分配权重的策略
# 初始分配权重
def initial_weights(stock_symbols):
    num_stocks = len(stock_symbols)
    return np.ones(num_stocks) / num_stocks

# 重新分配权重
def rebalance_weights(z_scores, date, initial_weight):
    ranked_stocks = z_scores.loc[date].sort_values()
    num_stocks = len(ranked_stocks)
    new_weights = np.zeros(num_stocks)
    
    # 根据排序结果重新分配权重
    for i, stock in enumerate(ranked_stocks.index):
        new_weights[i] = initial_weight * (1 + ranked_stocks.index.get_loc(stock) / num_stocks)
    new_weights /= new_weights.sum()  # 归一化权重

    return new_weights

def rebalance_rank_portfolio(stock_prices, window, initial_capital):
    transaction_cost=0.005
    lot_size=100
    z_scores = calculate_z_score(stock_prices, window)
    
    num_stocks = len(stock_prices.columns)
    initial_weight = 1 / num_stocks
    weights = initial_weights(stock_prices.columns)
    portfolio_value = initial_capital
    portfolio_values = [initial_capital]
    positions = np.zeros(num_stocks)
    cash = initial_capital
    buy_count = []
    sell_count = []
    annual_buy_count = 0
    annual_sell_count = 0
    current_year = stock_prices.index[window].year

    for date in stock_prices.index[window:]:
        if date.year != current_year:
            buy_count.append(annual_buy_count)
            sell_count.append(annual_sell_count)
            annual_buy_count = 0
            annual_sell_count = 0
            current_year = date.year
        
        # 根据Z-score排序重新分配权重
        new_weights = rebalance_weights(z_scores, date, initial_weight)
        
        # 根据新的权重进行买卖股票
        for i, stock in enumerate(stock_prices.columns):
            current_price = stock_prices[stock].loc[date]
            desired_position = portfolio_value * new_weights[i] // current_price
            if desired_position > positions[i]:
                buy_amount = (desired_position - positions[i]) // lot_size * lot_size
                if buy_amount > 0 and cash >= buy_amount * current_price * (1 + transaction_cost):
                    positions[i] += buy_amount
                    cash -= buy_amount * current_price * (1 + transaction_cost)
                    annual_buy_count += 1
            elif desired_position < positions[i]:
                sell_amount = (positions[i] - desired_position) // lot_size * lot_size
                if sell_amount > 0:
                    positions[i] -= sell_amount
                    cash += sell_amount * current_price * (1 - transaction_cost)
                    annual_sell_count += 1

        weights = new_weights
        portfolio_value = cash + np.sum(positions * stock_prices.loc[date])
        portfolio_values.append(portfolio_value)
    
    buy_count.append(annual_buy_count)
    sell_count.append(annual_sell_count)

    return portfolio_values, weights, buy_count, sell_count

In [85]:

def rebalance_portfolio(stock_prices, SSINDX, window, initial_capital, short_ma=None, long_ma=None):
    lot_size=100
    transaction_cost = 0.005
    z_scores = calculate_z_score(stock_prices, window)
    cov_matrix = stock_prices.pct_change().cov()
    mean_returns = stock_prices.pct_change().mean()
    dynamic_thresholds = dynamic_threshold(SSINDX, window)

    weights = portfolio_optimization(mean_returns, cov_matrix)
    portfolio_value = initial_capital
    portfolio_values = [initial_capital]
    positions = np.zeros(len(weights))
    cash = initial_capital
    buy_count = []
    sell_count = []
    annual_buy_count = 0
    annual_sell_count = 0
    current_year = stock_prices.index[window].year

    for date in stock_prices.index[window:]:
        # 获取当前交易日的动态阈值
        threshold = dynamic_thresholds.loc[date]
        if date.year != current_year:
            buy_count.append(annual_buy_count)
            sell_count.append(annual_sell_count)
            annual_buy_count = 0
            annual_sell_count = 0
            current_year = date.year
        
        # 根据Z-score和趋势调整权重
        new_weights = np.zeros(len(weights))
        for i, stock in enumerate(stock_prices.columns):
            if short_ma[stock].loc[date] > long_ma[stock].loc[date]:  # 确保在多头市场
                if z_scores[stock].loc[date] < -threshold:
                    new_weights[i] = weights[i] + 0.05
                elif z_scores[stock].loc[date] > threshold:
                    new_weights[i] = weights[i] - 0.05
                else:
                    new_weights[i] = weights[i]
            else:
                new_weights[i] = weights[i]  # 保持不变
        
        new_weights = np.clip(new_weights, 0, 1)
        new_weights = new_weights / np.sum(new_weights)
        
        # 根据新的权重进行买卖股票
        for i, stock in enumerate(stock_prices.columns):
            current_price = stock_prices[stock].loc[date]
            desired_position = portfolio_value * new_weights[i] // current_price
            if desired_position > positions[i]:
                buy_amount = (desired_position - positions[i]) // lot_size * lot_size
                if buy_amount > 0 and cash >= buy_amount * current_price * (1 + transaction_cost):
                    positions[i] += buy_amount
                    cash -= buy_amount * current_price * (1 + transaction_cost)
                    annual_buy_count += 1
            elif desired_position < positions[i]:
                sell_amount = (positions[i] - desired_position) // lot_size * lot_size
                if sell_amount > 0:
                    positions[i] -= sell_amount
                    cash += sell_amount * current_price * (1 - transaction_cost)
                    annual_sell_count += 1
    
        weights = new_weights
        portfolio_value = cash + np.sum(positions * stock_prices.loc[date])
        portfolio_values.append(portfolio_value)
    
    buy_count.append(annual_buy_count)
    sell_count.append(annual_sell_count)

    return portfolio_values, weights, buy_count, sell_count


In [86]:
# 读取Excel文件
df = pd.read_excel('Stocks.xlsx', sheet_name='Watchlist')

# 获取股票代码列表
stock_symbols = df['Stock Symbol'].tolist()
stock_names = df['Stock Name'].tolist()

# 定义时间范围
start_date = '2015-01-01'
end_date = '2024-08-08'

# 设置参数
initial_capital = 100000
window = 10
threshold = 2.5
transaction_cost = 0.005  # 假设交易成本为0.5%
rebalance_period = 7  # 调仓周期为3天

stock_data = get_stock_price(stock_symbols, start_date, end_date)
# 获取上证指数数据
SSINDX = yf.download('000001.SS', start=start_date, end=end_date)['Close']

# Ensure stock_data is a DataFrame and handle any missing values
stock_prices = pd.DataFrame(stock_data).ffill().bfill()
dates = stock_prices.index[window-1:]
short_window = 40
long_window = 200
short_ma, long_ma = calculate_moving_average(stock_prices, short_window, long_window)

# 调整rebalance_portfolio函数的调用
portfolio_values, final_weights, buy_count, sell_count = rebalance_portfolio(stock_prices, SSINDX, window, initial_capital, short_ma, long_ma)
portfolio_values1, final_weights1, buy_count1, sell_count1 = rebalance_rank_portfolio(stock_prices=stock_prices, window=window, initial_capital=initial_capital)
print(f'Final Portfolio Value: {portfolio_values[-1]}')
print(f'Final Portfolio Value: {portfolio_values1[-1]}')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Final Portfolio Value: 390115.50211018336
Final Portfolio Value: 424176.3171934486


In [87]:
# Total PnL Summary
summary_total = pnl_summary_total(portfolio_values, dates)
summary_total

/var/folders/2h/657ql8s954s35xlylnvyhmrw0000gn/T/ipykernel_84847/3691211594.py:90: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



,Total Return %,Sharpe Ratio,Max Drawdown %
0,290.12,0.7,42.74


In [88]:
summary_total1 = pnl_summary_total(portfolio_values1, dates)
summary_total1

/var/folders/2h/657ql8s954s35xlylnvyhmrw0000gn/T/ipykernel_84847/3691211594.py:90: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



,Total Return %,Sharpe Ratio,Max Drawdown %
0,324.18,0.76,38.34


In [89]:
# PnL Summary
summary = pnl_summary(portfolio_values, dates, buy_count, sell_count)
summary

,Start Value,End Value,Return Std,Annual Return %,Sharpe Ratio,Max Drawdown %,Buy Count,Sell Count
Year,,,,,,,,
2015,100000.00,153213.31,0.03,53.21,1.27,38.94,222,180
2016,142238.24,146346.59,0.02,2.89,0.11,22.55,150,131
2017,147102.77,168896.81,0.01,14.82,0.81,17.18,93,78
2018,171178.99,126630.04,0.01,-26.02,-1.23,36.12,95,93
2019,126140.28,160997.06,0.01,27.63,1.31,14.88,139,120
2020,163744.94,213211.91,0.02,30.21,1.22,19.32,176,136
2021,222208.03,360082.99,0.02,62.05,2.23,20.73,301,250
2022,358286.73,372143.73,0.02,3.87,0.15,18.84,322,294
2023,380775.62,367749.70,0.01,-3.42,-0.20,21.14,209,187


In [90]:
# PnL Summary
summary1 = pnl_summary(portfolio_values1, dates, buy_count1, sell_count1)
summary1

,Start Value,End Value,Return Std,Annual Return %,Sharpe Ratio,Max Drawdown %,Buy Count,Sell Count
Year,,,,,,,,
2015,100000.00,155948.73,0.03,55.95,1.36,37.64,218,165
2016,144259.48,145981.87,0.02,1.19,0.04,23.15,100,92
2017,146611.62,177841.90,0.01,21.30,1.28,13.96,90,71
2018,179790.58,142154.91,0.01,-20.93,-1.02,32.07,102,92
2019,141742.52,177024.86,0.01,24.89,1.22,17.07,106,92
2020,179691.54,225643.99,0.02,25.57,1.05,18.94,178,141
2021,233039.07,347237.73,0.02,49.00,1.90,19.85,230,187
2022,345156.45,344855.32,0.02,-0.09,-0.00,20.19,242,226
2023,351944.27,373776.67,0.01,6.20,0.42,12.56,157,177


In [91]:
# Interactive Plot for Portfolio Value
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=portfolio_values, mode='lines', name='Portfolio Value'))
fig.update_layout(title='Portfolio Value Over Time', xaxis_title='Date', yaxis_title='Portfolio Value')
fig.show()

In [92]:
# Interactive Plot for Portfolio Value
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=portfolio_values1, mode='lines', name='Portfolio Value'))
fig.update_layout(title='Portfolio Value Over Time', xaxis_title='Date', yaxis_title='Portfolio Value')
fig.show()